# Kalman Filter Walkthrough

This notebook will walk through the Kalman Filter (KF) algorithm. It is a simple state-space model which makes some Gaussian assumptions about the transition dynamics and the emission dynamics. This results in a very simple model to use because all of the important quantities of interest (e.g. filtering, smoothing, and forecasting) are available in closed-form.

**Resources**:

* 

## Model

**Transition Model**

This model describes the transition dynamics between the different states at time $t$ and $t+1$.

$$
\mathbf{z}_{t} = \mathbf{A}_t \mathbf{z}_{t-1} + \mathbf{Q}_t
$$

Notice how it assumes a linear transition with an additive Gaussin noise assumption, $\boldsymbol{\epsilon}_{\mathbf{z}_t} \sim \mathcal{N}(\mathbf{0},\mathbf{Q}_t)$. We can also rewrite this as a conditional distribution.

$$
p(\mathbf{z}_{t}|\mathbf{z}_{t-1}) = \mathcal{N}(\mathbf{z}_t|\mathbf{A}_t\mathbf{z}_{t-1}, \mathbf{Q}_t)
$$

---
**Observation Model**

$$
\mathbf{x}_t = \mathbf{C}_t\mathbf{z}_t + \mathbf{R}_t
$$

where can also rewrite this as a distribution:

$$
p(\mathbf{x}_t|\mathbf{z}_t) = \mathcal{N}(\mathbf{x}_t|\mathbf{C}_t\mathbf{z}_t, \mathbf{R}_t)
$$


---
**Free Parameters**

In this case, we have the following free parameters, $\boldsymbol{\theta}_t$, in this model:

$$
\boldsymbol{\theta}_t = \left\{\mathbf{A}_t, \mathbf{Q}_t, \mathbf{C}_t, \mathbf{R}_t \right\}
$$

**Note**: we can assume the parameters are independent of time, $t$, which would provide us with a simpler model. This is called a *stationary* assumption which may be cheaper, but also invalid in some cases.

In [1]:
import numpy as np
from typing import NamedTuple
from dataclasses import dataclass

### Distribution

Kalman Filter Portions:

* $A_t \in \mathbb{R}^{N_\mathbf{z} \times N_\mathbf{z}}$
* $R_t$ = `N_z x N_z` 
* $C_t$ = `N_x x N_z`
* $Q_t$ = `N_x x N_x`

In [2]:
@dataclass
class KFDistribution:
    transition_matrix : np.ndarray
    transition_noise : np.ndarray
    measurement_matrix : np.ndarray
    measurement_noise : np.ndarray
    
    def predictive_mean(self, x: np.ndarray) -> np.ndarray:
        return predictive_mean(x, self.measurement_matrix)
    
    def predictive_cov(self, cov: np.ndarray) -> np.ndarray:
        return predictive_cov(cov, self.transition_matrix, self.transition_noise)

### State

In [3]:
class State(NamedTuple):
    mean : np.ndarray
    cov : np.ndarray
    

### Transition Model

### Observation Model

### Predict Step

$$
\begin{aligned}
\boldsymbol{\mu}_t &= \mathbf{A}_{t-1}\boldsymbol{\mu}_{t-1} \\
\boldsymbol{\Sigma}_t &= \mathbf{A}_{t-1}\boldsymbol{\Sigma}_{t-1}\mathbf{A}_t^\top + \mathbf{Q}_t
\end{aligned}
$$

where:
* `KalmanDist` $= \mathcal{N}(\boldsymbol{\mu}_t, \boldsymbol{\Sigma}_t)$.

In [4]:
def predict_step(state: State, dist: KFDistribution) -> State:
    """Prediction step in Kalman filter eqns"""
    # predictive mean, µ = F µ
    mean = dist.predictive_mean(state.mean)
    
    # predictive covariance, Σ = F Σ F' + Q
    cov = dist.predictive_cov(state.cov)
    
    return mean, cov

def predictive_mean(mean: np.ndarray, transition: np.ndarray) -> np.ndarray:
    """predictive mean in update step
    
    µ = F µ
    """
    return transition @ mean

def predictive_cov(cov: np.ndarray, transition: np.ndarray, noise: np.ndarray) -> np.ndarray:
    """predictive covariance in update step
    
    Σ = F Σ F' + Q
    """
    return transition @ cov @ transition.T + noise

### Update Step

This is the measurement step which we can compute using Bayes rule like so.

$$
p(\mathbf{z}_t|\mathbf{x}_t,\mathbf{x}_{1:t-1}) \propto p(\mathbf{x}_t|\mathbf{z}_t)p(\mathbf{z}_t|\mathbf{z}_{1:t-1})
$$

This quantity is given by:

$$
\begin{aligned}
p(\mathbf{z}_t|\mathbf{x}_{1:t}) &= \mathcal{N}(\mathbf{z}_t|\boldsymbol{\mu}_t, \boldsymbol{\Sigma}_t)
\end{aligned}
$$

This is the dist. for the update step and it is given by these equations:

$$
\begin{aligned}
\boldsymbol{\mu}_t &= \boldsymbol{\mu}_{t|t-1} + \mathbf{K}_t\mathbf{r}_t \\
\boldsymbol{\Sigma}_t &= \left( \mathbf{I} - \mathbf{K}_t\mathbf{C}_t \right)\boldsymbol{\Sigma}_{t|t-1}
\end{aligned}
$$

where:
* $K_t$ - Kalman Gain Matrix
* $r_t$ - innovation/residual

---
**Residual**

This quantity is the difference between our predicted observations and the actual observations. This is given by:

$$
\begin{aligned}
\mathbf{r}_t &= \mathbf{x}_t - \hat{\mathbf x}_t \\
\hat{\mathbf x}_t &= \mathbf{C}_t {\boldsymbol \mu}_{t|t-1}
\end{aligned}
$$

where:

* $C_t$ - measurement matrix
* ${\boldsymbol \mu}_{t|t-1}$ - mean from the predict step

In [7]:

def residual(obs: np.ndarray, state: State, dist: KFDistribution) -> np.ndarray:
    
    # unroll variables
    µ = state.mean
    A = dist.transition_matrix
    C = dist.measurement_matrix
    
    # predictive mean
    µ_pred = dist.predictive_mean(state.mean)
    
    # difference
    obs_pred = C @ µ_pred
    
    # residual
    res = obs - obs_pred
    
    return res

---
**Kalman Gain Matrix**

$$
\begin{aligned}
\mathbf{K}_t &= \boldsymbol{\Sigma}_{t|t-1}C_t^\top \mathbf{S}_t^{-1} \\
&= \mathbf{C}_t \boldsymbol{\Sigma}_{t|t-1}\mathbf{C}_t^\top + \mathbf{R}_t
\end{aligned}
$$

where can also use the matrix inversion lemma and rewrite the Kalman gain matrix.

$$
\begin{aligned}
\mathbf{K}_t &= \boldsymbol{\Sigma}_{t|t-1} \mathbf{C}^\top \left( \mathbf{C}\boldsymbol{\Sigma}_{t|t-1} \mathbf{C}^\top + \mathbf{R} \right)^{-1} \\
&= \left( \boldsymbol{\Sigma}_{t|t-1}^{-1} + \mathbf{C}^\top \mathbf{RC}\right)^{-1}\mathbf{C}^\top \mathbf{R}^{-1}
\end{aligned}
$$

We may get some computational gains this way because we could use the sparsity of the precision matrix, $\boldsymbol \Sigma^{-1}$, instead of the covariance matrix, $\boldsymbol \Sigma$. For now, we can use the standard method.

In [9]:
def kalman_gain_matrix(state: State, dist: KFDistribution)-> np.ndarray:
    
    # unroll variables
    A = dist.transition_matrix
    Q = dist.transition_noise
    C = dist.measurement_matrix
    R = dist.measurement_noise
    𝚺 = state.cov
    
    # predictive covariance
    𝚺 = dist.predictive_cov(𝚺)
    
    # kalman gain
    K = 𝚺 @ C.T @ np.linalg.inv(C @ 𝚺 @ C.T + R)
    
    return K

---
**Update Step**

$$
\begin{aligned}
p(\mathbf{z}_t|\mathbf{x}_{1:t}) &= \mathcal{N}(\mathbf{z}_t|\boldsymbol{\mu}_t, \boldsymbol{\Sigma}_t) \\
\boldsymbol{\mu}_t &= \boldsymbol{\mu}_{t|t-1} + \mathbf{K}_t\mathbf{r}_t \\
\boldsymbol{\Sigma}_t &= \left( \mathbf{I} - \mathbf{K}_t\mathbf{C}_t \right)\boldsymbol{\Sigma}_{t|t-1}
\end{aligned}
$$

where:
* $K_t$ - *Kalman Gain Matrix*
* $r_t$ - *innovation/residual*

In [12]:
def update_step(
    obs: np.ndarray,
    state: State, 
    dist: KFDistribution,
) -> State:
    
    # unroll variables
    C = dist.measurement_matrix
    I = np.eye(C.shape[0])
    µ = state.mean
    𝚺 = state.cov
    
    # predict step
    µ = dist.predictive_mean(µ)
    𝚺 = dist.predictive_cov(𝚺)
    
    # kalman gain
    K = kalman_gain_matrix(state, dist)
    
    # innovation/residual
    r = residual(obs, state, dist)
    
    # update step
    µ = µ + K @ r
    𝚺 = (I - K @ C) @ 𝚺
    
    return µ, 𝚺

## Marginal Likelihood

We can compute the log-likelihood of the sequence, i.e. the likelihood that the parameters given fit the data seen/trained-on. This allows us to actually train the Kalman Filter given the data. The equation is given by:

$$
p(\mathbf{x}_t) = \sum_t \log p(\mathbf{x}_t|\mathbf{x}_{1:t-1})
$$

We can write this probability distribution exactly because we know it is Gaussian distributed

$$
p(\mathbf{x}_t|\mathbf{x}_{1:t-1})= \mathcal{N}(\mathbf{C}_t \boldsymbol{\mu}_{t|t-1}, \mathbf{S}_t)
$$

where:
*  ${\boldsymbol \mu}_{t:t-1}$ - is the mean
* 


## Posterior Predictive

This is a one-step-ahead predictive density for the observations. It predicts the next time step using all of the previous observations. It is given by these equations:

$$
p(\mathbf{x}_t|\mathbf{x}_{1:t-1}) = \int \mathcal{N}(\mathbf{x}_t|\mathbf{Cz}_t, \mathbf{R})\mathcal{N}(\mathbf{z}_t|\boldsymbol{\mu}_{t|t-1},\boldsymbol{\Sigma}_{t|t-1})d\mathbf{z}_t
$$

In [13]:
def forecast():
    return None

## Smoothing Algorithm

This is a message passing algorithm that propagates from right to left after everything has been observed.

$$
\begin{aligned}
p(\mathbf{z}_t|\mathbf{x}_{1:T}) &= \mathcal{N}(\mathbf{z}_t|\boldsymbol{\mu}_{t:T},\boldsymbol{\Sigma}_{t:T})
\end{aligned}
$$

This is given by these equations:

$$
\begin{aligned}
\boldsymbol{\mu}_{t|T} &= \boldsymbol{\mu}_{t|t} + \mathbf{J}_t \left(\boldsymbol{\mu}_{t+1|T} - \boldsymbol{\mu}_{t+1|t} \right) \\
\boldsymbol{\Sigma}_{t|T} &= \boldsymbol{\Sigma}_{t|t} + \mathbf{J}_t \left(\boldsymbol{\mu}_{t+1|T} - \boldsymbol{\mu}_{t+1|t} \right) \mathbf{J}_t^\top \\
\mathbf{J}_t &= \boldsymbol{\Sigma}_{t|t} \mathbf{A}_{t+1}^\top \boldsymbol{\Sigma}_{t+1|t}^{-1}
\end{aligned}
$$


In [14]:
def rts_smoother(state, dist):
    return None

## Model Class

In [ ]:
def LGSSM:
    def __init__(self):
        pass
    
    def log_prob(self, x: np.ndarray) -> np.ndarray:
        pass
    
    def sample(self):
        pass
    
    def filter(self):
        pass
    
    def update(self):
        pass
    
    def predict(self):
        pass
    
    def smooth(self):
        pass
    

## Application I - Object Tracking


$$
\mathbf{z}_t = \mathbf{A}_t \mathbf{z}_{t-1} + \boldsymbol{\epsilon}_t
$$

We can explicitly define the system dynamics as so:

$$
\begin{bmatrix}
z^1_t \\ z^2_t \\ z^3_t \\ z^4_t
\end{bmatrix}
$$

## Application II - Predator Prey Model

In this example, we are going to assume that we have a 3d vector, $\mathbf{x} = \{x^0, x^1, x^2\}$, which describes the dynamics. The dynamics between every time step is given by:

$$
\begin{aligned}
\frac{dx^0}{dt} &= -0.1x^0 + 0.2x^1 \\
\frac{dx^1}{dt} &= -0.2x^0 + x^2 \\
\frac{dx^2}{dt} &= 0
\end{aligned}
$$

---
We can rewrite this as a discretized version like so:

$$
\mathbf{x}_{t+1} = \mathbf{F} \mathbf{x}_t
$$

where: $\mathbf{F}=\begin{bmatrix} -0.1 & 0.2 & 0 \\ -0.2 & 0 & 1 \\ 0 & 0 & 0 \end{bmatrix}$.
